## CLASS BASED MODEL
#### INPUT THE SUBJECT, LESSONS PER WEEK, AND TEACHER 

In [9]:
from ortools.sat.python import cp_model

# Define the school structure
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
periods_per_day = 6
total_periods = len(days) * periods_per_day

# Updated class structure with teacher assignments
classes = {
    "Grade 4A": [
        ("Math", 5, "Alice"),
        ("English", 5, "Bob"),
        ("Science", 4, "Dennis"),
        ("History", 3, "Eric")
    ],
    "Grade 4B": [
        ("Math", 5, "Alice"),
        ("English", 5, "Bob"),
        ("Geography", 4, "Eric"),
        ("Biology", 3, "Dennis")
    ],
    "Grade 5C": [
        ("Math", 5, "Alice"),
        ("English", 5, "Bob"),
        ("History", 4, "Eric")
    ],
    "Grade 6D": [
        ("Math", 5, "Charlie"),
        ("English", 5, "Bob"),
        ("Physics", 4, "Dennis"),
        ("Economics", 4, "Fred")
    ],
    "Grade 7E": [
        ("Math", 5, "Charlie"),
        ("English", 5, "Bob"),
        ("Business Studies", 5, "Fred")
    ]
}

# Create the CP model
model = cp_model.CpModel()

# Decision variables: (class, subject, day, period) -> 1 if scheduled, 0 otherwise
schedule = {}
for class_name, subjects in classes.items():
    for subject, weekly_hours, teacher in subjects:
        for day in days:
            for period in range(periods_per_day):
                schedule[(class_name, subject, day, period)] = model.NewBoolVar(
                    f"{class_name}_{subject}_{day}_P{period}"
                )

# Constraint: Allocate required teaching hours per subject
for class_name, subjects in classes.items():
    for subject, weekly_hours, teacher in subjects:
        model.Add(
            sum(schedule[(class_name, subject, day, period)]
                for day in days for period in range(periods_per_day)) == weekly_hours
        )

# Constraint: No subject appears more than once per day for the same class
for class_name, subjects in classes.items():
    for subject, _, teacher in subjects:
        for day in days:
            model.Add(
                sum(schedule[(class_name, subject, day, period)]
                    for period in range(periods_per_day)) <= 1
            )

# Assign teachers based on the subjects they teach
teacher_schedule = {}
for class_name, subjects in classes.items():
    for subject, _, teacher in subjects:
        for day in days:
            for period in range(periods_per_day):
                teacher_schedule[(teacher, subject, class_name, day, period)] = model.NewBoolVar(
                    f"{teacher}_{subject}_{class_name}_{day}_P{period}"
                )

# Constraint: A teacher can't teach two classes at the same time
for teacher in {t for s in classes.values() for _, _, t in s}:  # Extract unique teachers
    for day in days:
        for period in range(periods_per_day):
            model.Add(
                sum(teacher_schedule[(teacher, subject, class_name, day, period)]
                    for class_name, subjects in classes.items()
                    for subject, _, t in subjects if t == teacher) <= 1
            )

# Ensure a teacher only teaches when their subject is scheduled
for class_name, subjects in classes.items():
    for subject, _, teacher in subjects:
        for day in days:
            for period in range(periods_per_day):
                model.Add(
                    teacher_schedule[(teacher, subject, class_name, day, period)] ==
                    schedule[(class_name, subject, day, period)]
                )

# Ensure all periods are filled for each class (flexible constraint)
for class_name in classes:
    for day in days:
        for period in range(periods_per_day):
            model.Add(
                sum(schedule[(class_name, subject, day, period)]
                    for subject, _, _ in classes[class_name]) <= 1
            )

# Solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print the class timetables
if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
    print("\n--- Class Timetables ---")
    for class_name in classes:
        print(f"\n{class_name} Timetable:")
        for day in days:
            print(f"\n{day}:")
            for period in range(periods_per_day):
                for subject, _, teacher in classes[class_name]:
                    if solver.Value(schedule[(class_name, subject, day, period)]):
                        print(f"  Period {period+1}: {subject} (Taught by {teacher})")

    # Print the teacher timetables
    print("\n--- Teacher Timetables ---")
    for teacher in {t for s in classes.values() for _, _, t in s}:  # Extract unique teachers
        print(f"\n{teacher}'s Timetable:")
        for day in days:
            print(f"\n{day}:")
            for period in range(periods_per_day):
                for class_name, subjects in classes.items():
                    for subject, _, t in subjects:
                        if t == teacher and solver.Value(teacher_schedule[(teacher, subject, class_name, day, period)]):
                            print(f"  Period {period+1}: {subject} ({class_name})")
else:
    print("No feasible timetable found.")



--- Class Timetables ---

Grade 4A Timetable:

Monday:
  Period 1: Science (Taught by Dennis)
  Period 2: Math (Taught by Alice)
  Period 4: History (Taught by Eric)
  Period 5: English (Taught by Bob)

Tuesday:
  Period 1: History (Taught by Eric)
  Period 3: Science (Taught by Dennis)
  Period 5: English (Taught by Bob)
  Period 6: Math (Taught by Alice)

Wednesday:
  Period 2: History (Taught by Eric)
  Period 5: English (Taught by Bob)
  Period 6: Math (Taught by Alice)

Thursday:
  Period 3: Science (Taught by Dennis)
  Period 5: English (Taught by Bob)
  Period 6: Math (Taught by Alice)

Friday:
  Period 4: Science (Taught by Dennis)
  Period 5: English (Taught by Bob)
  Period 6: Math (Taught by Alice)

Grade 4B Timetable:

Monday:
  Period 1: English (Taught by Bob)
  Period 2: Geography (Taught by Eric)
  Period 4: Biology (Taught by Dennis)
  Period 6: Math (Taught by Alice)

Tuesday:
  Period 1: English (Taught by Bob)
  Period 3: Geography (Taught by Eric)
  Period 5: Math

## TEACHER BASED MODEL
#### INPUT THE SUBJECT, LESSONS PER WEEK, AND CLASS TAUGHT 

In [10]:
from ortools.sat.python import cp_model

# Define the school structure
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
periods_per_day = 6
total_periods = len(days) * periods_per_day

# Updated teacher structure with class assignments
teachers = {
    "Alice": [
        ("Math", 5, "Grade 4A"),
        ("Math", 5, "Grade 4B"),
        ("Math", 5, "Grade 5C")
    ],
    "Bob": [
        ("English", 5, "Grade 4A"),
        ("English", 5, "Grade 4B"),
        ("English", 5, "Grade 5C"),
        ("English", 5, "Grade 6D"),
        ("English", 5, "Grade 7E")
    ],
    "Charlie": [
        ("Math", 5, "Grade 6D"),
        ("Math", 5, "Grade 7E")
    ],
    "Dennis": [
        ("Science", 4, "Grade 4A"),
        ("Biology", 3, "Grade 4B"),
        ("Physics", 4, "Grade 6D")
    ],
    "Eric": [
        ("History", 3, "Grade 4A"),
        ("Geography", 4, "Grade 4B"),
        ("History", 4, "Grade 5C")
    ],
    "Fred": [
        ("Economics", 4, "Grade 6D"),
        ("Business Studies", 5, "Grade 7E")
    ]
}

# Create the CP model
model = cp_model.CpModel()

# Decision variables: (teacher, subject, class, day, period) -> 1 if scheduled, 0 otherwise
schedule = {}
for teacher, assignments in teachers.items():
    for subject, weekly_hours, class_name in assignments:
        for day in days:
            for period in range(periods_per_day):
                schedule[(teacher, subject, class_name, day, period)] = model.NewBoolVar(
                    f"{teacher}_{subject}_{class_name}_{day}_P{period}"
                )

# Constraint: Allocate required teaching hours per subject
for teacher, assignments in teachers.items():
    for subject, weekly_hours, class_name in assignments:
        model.Add(
            sum(schedule[(teacher, subject, class_name, day, period)]
                for day in days for period in range(periods_per_day)) == weekly_hours
        )

# Constraint: No subject appears more than once per day for the same class
for teacher, assignments in teachers.items():
    for subject, _, class_name in assignments:
        for day in days:
            model.Add(
                sum(schedule[(teacher, subject, class_name, day, period)]
                    for period in range(periods_per_day)) <= 1
            )

# Constraint: A teacher cannot be in multiple places at once
for teacher in teachers:
    for day in days:
        for period in range(periods_per_day):
            model.Add(
                sum(schedule[(teacher, subject, class_name, day, period)]
                    for subject, _, class_name in teachers[teacher]) <= 1
            )

# Ensure all periods are filled for each class (flexible constraint)
for class_name in {c for assignments in teachers.values() for _, _, c in assignments}:
    for day in days:
        for period in range(periods_per_day):
            model.Add(
                sum(schedule[(teacher, subject, class_name, day, period)]
                    for teacher, assignments in teachers.items()
                    for subject, _, c in assignments if c == class_name) <= 1
            )

# Solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print the teacher timetables
if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
    print("\n--- Teacher Timetables ---")
    for teacher in teachers:
        print(f"\n{teacher}'s Timetable:")
        for day in days:
            print(f"\n{day}:")
            for period in range(periods_per_day):
                for subject, _, class_name in teachers[teacher]:
                    if solver.Value(schedule[(teacher, subject, class_name, day, period)]):
                        print(f"  Period {period+1}: {subject} ({class_name})")

    # Print the class timetables
    print("\n--- Class Timetables ---")
    for class_name in {c for assignments in teachers.values() for _, _, c in assignments}:
        print(f"\n{class_name} Timetable:")
        for day in days:
            print(f"\n{day}:")
            for period in range(periods_per_day):
                for teacher, assignments in teachers.items():
                    for subject, _, c in assignments:
                        if c == class_name and solver.Value(schedule[(teacher, subject, class_name, day, period)]):
                            print(f"  Period {period+1}: {subject} (Taught by {teacher})")
else:
    print("No feasible timetable found.")



--- Teacher Timetables ---

Alice's Timetable:

Monday:
  Period 1: Math (Grade 4B)
  Period 2: Math (Grade 5C)
  Period 5: Math (Grade 4A)

Tuesday:
  Period 2: Math (Grade 5C)
  Period 4: Math (Grade 4A)
  Period 5: Math (Grade 4B)

Wednesday:
  Period 4: Math (Grade 4B)
  Period 5: Math (Grade 5C)
  Period 6: Math (Grade 4A)

Thursday:
  Period 3: Math (Grade 5C)
  Period 4: Math (Grade 4B)
  Period 6: Math (Grade 4A)

Friday:
  Period 4: Math (Grade 4B)
  Period 5: Math (Grade 5C)
  Period 6: Math (Grade 4A)

Bob's Timetable:

Monday:
  Period 2: English (Grade 4B)
  Period 3: English (Grade 4A)
  Period 4: English (Grade 5C)
  Period 5: English (Grade 6D)
  Period 6: English (Grade 7E)

Tuesday:
  Period 1: English (Grade 7E)
  Period 2: English (Grade 4B)
  Period 4: English (Grade 5C)
  Period 5: English (Grade 6D)
  Period 6: English (Grade 4A)

Wednesday:
  Period 1: English (Grade 5C)
  Period 2: English (Grade 4B)
  Period 3: English (Grade 4A)
  Period 4: English (Grade 7E